This notebook processes the PPC output to get the widths on Forier Parameters: <br>
1) parses the PPC output to get the LLHs for all Doms and shifts <br>
2) generate the file for total LLH for each perturbed ice model and get the width for each Fourier Parameter. <br>
3) plot the total LLH vs shift for each Fourier mode.

In [96]:
import numpy as np 
import glob
import os
import re

import pylab
from scipy.interpolate import interp1d
from scipy.optimize import brentq
import subprocess

from numpy.polynomial import polynomial
%matplotlib inline

# Parsing the ppc output 

In [ ]:
def Processoutput(rootdir, outpath):
    count=0
    PrintEvery=100
    fit_dirs = []
    fit_files = glob.glob(rootdir + "/out*")
    if not fit_files :
        print("WARNING: No Fit Directories Found In " + str(rootdir) )
        return
    else:
        print("Found "  + str(len(fit_files)) + " Fit Files....")


    outfile = open(outpath, "a")

    print("Writing Output to: "  + str(outfile))
    outfile.write("#Param-mode shift DOM LLH \n")

    for fit_filen in fit_files:
        try:
            fit_file = open(str(fit_filen),'r')
        except:
            print("WARNING: " + str(fit_filen) + " Does Not Exist!" )
        else:
            try:
                lines= fit_file.readlines()     
            except:
                print("WARNING: " + str(fit_filen) + " Is Empty!" )
                break
            HeaderLine=lines[2]
            Param=HeaderLine.split("_")[0].split(" ")[1]
            ParamNum=HeaderLine.split("_")[1]
            try:
                shift=float(HeaderLine.split("_")[2].replace(".dat",""))
                #print("sh",shift)
            except:
                print("Shift not found")
            for line in lines[3:-1]:
                if "DOM " in line:
                    if not "Bad" in line:
                        DOM=int(line.split()[1])
                        
                if 'LLH' in line:
                    LLH=re.findall("LLH=\d+\.\d+", line)[0][4:]
                    DT=line.split("=")[2][0:-3]
                    if(DOM!=0):
                        writeline=str(Param)+"-"+str(ParamNum)+" " +str(shift)+ " " + str(DOM) + " " + str(LLH) 
                        outfile.write(writeline + "\n")                    
                    count=count+1
                    #if(count%PrintEvery==0):
                        #print("Absorbed "+str(count) + " LLHs \n")
        fit_file.close()
    outfile.close()

    print("Done!")

In [ ]:
#Amp-0:
Processoutput("/data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/Amp/0/out/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/Amp_0.txt")

In [ ]:
#Phs-6:
Processoutput("/data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/Phs/6/out/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/Phs_6.txt")

In [ ]:
Params=["Amp","Phs"]     #Forier Parameter (eg. Amp or Phs)
modes=[1,2,3,4,5]       # mode (eg. 0,1,2,3...) no mode '0' for Phs
for Param in Params:
    for Param_num in modes:
        outdir='/data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/'+str(Param)+"/"+str(Param_num)+"/out/"
        outfile='/data/user/anegi/ppc/LLHs/spice_ftp-v2/'+str(Param)+"_"+str(Param_num)+".txt"
        Processoutput(outdir,outfile)
        

# Getting widths from the output LLHs: 

In [4]:
def LLHWidths(infile,outdir_llh,outdir_widths,Bad_Doms=[]):
    dat=np.loadtxt(infile,dtype=str)
    params=np.unique(dat[:,0])
    for param in params:
        LLHDict=dict()
        LLHDict[str(param)]=dict()
        mask1=(dat[:,0]==str(param))
        dat2=dat[mask1,1:]        
        shifts=np.unique(dat2[:,0])
      #  print(len(shifts))
        pairs=[0,0]
        for shift in shifts:
            mask=(dat[:,1]==shift)
            subdat=dat[mask,1:]         #subarray for a particular shift value
            #print(np.shape(subdat))
            #print(subdat)
            LLH_shift=0
            BadDom=[]        
            for i in range(1,61):
                if i not in Bad_Doms:
                    try:
                        LLH_dom=float(subdat[subdat[:,0]==str(i),1])      #DoM=i  ,LLH(i)=LLH_dom
                        LLH_shift+=LLH_dom
                        #print(LLH_dom,i)
                    except:
                        #print("shift ",shift,"missing LLH: DoM",i)
                        BadDom.append(i)
                        pass        
            if len(BadDom)<2:
                LLH_shift=LLH_shift*(60-len(Bad_Doms))/(60-len(Bad_Doms)-len(BadDom))                   #rescaling total LLH if only missing 1 DoM
                LLHDict[str(param)][float(shift)]=float(LLH_shift)
            else:                                #ignoring the shift if missing LLh for more than 1 DoM (rerun jobs for missing DoMs if required)
                print("shift ",shift,"missing LLH: DoM",BadDom)           
#        print(LLHDict)
        width=[]
        for Param in LLHDict.keys():
            Shifts_dict=LLHDict[Param]
            shift_llh=np.array(list(Shifts_dict.items()))
            print("\n")
            #print(shift_llh)
            if os.path.isfile(str(outdir_llh)+"/"+str(Param.split("-")[0])+"_"+str(Param.split("-")[1])+"_llh.txt")==0:
                print("creating new llh file for ",Param)
                np.savetxt(str(outdir_llh)+"/"+str(Param.split("-")[0])+"_"+str(Param.split("-")[1])+"_llh.txt",shift_llh,delimiter=" ")
            else:
                print("llh file already exists")
            shifts=shift_llh[:,0]
            llhs=shift_llh[:,1]
            llh_min=min(llhs)
            #print(llh_min)
            delta_llhs=llhs-llh_min
            #print(delta_llhs)
            X=[]
            Y=[]
            for i in range(0,len(shifts)):
                if delta_llhs[i]<5:                      #only taking shifts withh LLH<5 for the curve fitting 
                    Y.append(delta_llhs[i])
                    X.append(shifts[i])
            FittedCurve=polynomial.polyfit(X,Y,2)        #fitting to a quadratic polynomial
            vals=np.arange(-3,3,0.001)
            minval=np.min(polynomial.polyval(vals,FittedCurve))
            PlusOneSig=(-FittedCurve[1]+np.sqrt(FittedCurve[1]**2-4.*(-0.5-minval+FittedCurve[0])*FittedCurve[2]))/(2.*FittedCurve[2])
            MinusOneSig=(-FittedCurve[1]-np.sqrt(FittedCurve[1]**2-4.*(-0.5-minval+FittedCurve[0])*FittedCurve[2]))/(2.*FittedCurve[2])
            Zero=(PlusOneSig+MinusOneSig)/2.
                   
            if os.path.isfile(str(outdir_widths)+"/"+str(Param.split("-")[0])+"_widths.txt")==1:
                outfile = open(str(outdir_widths)+"/"+str(Param.split("-")[0])+"_widths.txt", "r")
                lines= outfile.readlines()
                count=0
                for line in lines:
                    if line[0:3]==str(Param.split("-")[0]) and line[4:5]==str(Param.split("-")[1]):  
                        count+=1
                outfile.close()        
                if count!=0:
                    print("Parameter width already exists: ", Param)
                else:
                    print("writing Param width: ",Param)
                    outfile = open(str(outdir_widths)+"/"+str(Param.split("-")[0])+"_widths.txt", "a")
                    writeline=str(Param.split("-")[0])+" "+str(Param.split("-")[1])+" "+str(Zero)+" "+str(MinusOneSig)+" "+str(PlusOneSig)                
                    outfile.write(writeline + "\n")
                    outfile.close()
            else: 
                print("creating new width file: ", str(outdir_widths)+"/"+str(Param.split("-")[0])+"_widths.txt")
                outfile = open(str(outdir_widths)+"/"+str(Param.split("-")[0])+"_widths.txt", "w")
                writeline=str(Param.split("-")[0])+" "+str(Param.split("-")[1])+" "+str(Zero)+" "+str(MinusOneSig)+" "+str(PlusOneSig)                
                outfile.write(writeline + "\n")
                outfile.close()


In [ ]:
#Amp_0

LLHWidths("/data/user/anegi/ppc/LLHs/spice_ftp-v2/Amp_0.txt","/data/user/anegi/ppc/LLHs/spice_ftp-v2/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/",Bad_Doms=[])

In [ ]:
outdir="/data/user/anegi/ppc/LLHs/spice_ftp-v2"
Params=["Amp","Phs"]     #Forier Parameter (eg. Amp or Phs)
modes=[1,2,4,5]       # mode (eg. 0,1,2,3...) no mode '0' for Phs
for Param in Params:
    for Param_num in modes:
        infile=str(outdir)+"_run_2/"+str(Param)+"_"+str(Param_num)+".txt"
        LLHWidths(infile,outdir,outdir)

In [ ]:
#Phs_6

LLHWidths("/data/user/anegi/ppc/LLHs/spice_ftp-v2/Phs_6.txt","/data/user/anegi/ppc/LLHs/spice_ftp-v2/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/",Bad_Doms=[])

# Ploting the LLH vs Shift for the given Fourier Parameter

In [ ]:
Param="Amp"
mode=1
llh_dir="/data/user/anegi/ppc/LLHs/spice_ftp-v2/"
if os.path.exists((str(llh_dir)+"/"+str(Param)+"_"+str(mode)+"_llh.txt"))==True:
    data=np.loadtxt(str(llh_dir)+"/"+str(Param)+"_"+str(mode)+"_llh.txt")
    shift=data[:,0]
    llhs=data[:,1]
    delta_llh=llhs-min(llhs)
    shifts=[]
    delta_llhs=[]
    for i in range(0,len(shift)):
        #print(delta_llh[i,1])
        if delta_llh[i]<5:
            delta_llhs.append(data[i,1])
            shifts.append(data[i,0])
        #if len(shifts.append)<5:
            
            
    print(shifts)        
    vals=np.arange(-.3,.3,0.001)
    FittedCurve=polynomial.polyfit(shifts,delta_llhs,2)        #fitting to a quadratic polynomial
    minval=np.min(polynomial.polyval(vals,FittedCurve))  
    PlusOneSig=(-FittedCurve[1]+np.sqrt(FittedCurve[1]**2-4.*(-0.5-minval+FittedCurve[0])*FittedCurve[2]))/(2.*FittedCurve[2])
    MinusOneSig=(-FittedCurve[1]-np.sqrt(FittedCurve[1]**2-4.*(-0.5-minval+FittedCurve[0])*FittedCurve[2]))/(2.*FittedCurve[2])
    Zero=(PlusOneSig+MinusOneSig)/2.
    print("2*Width= "PlusOneSig-MinusOneSig)

    pylab.figure(figsize=(3,3))
    fitcurve=polynomial.polyval(vals,FittedCurve)
    pylab.plot(vals,fitcurve-min(fitcurve),color='DarkRed',linewidth=2)
    pylab.plot(shifts,delta_llhs-min(fitcurve),'o',color='red')
    pylab.hlines(y=0.5,xmin= MinusOneSig,xmax=PlusOneSig,linestyle='solid', color='blue')
    pylab.xlabel("Shift")
    pylab.ylabel("DLLH")
    pylab.title(str(Param)+"_"+str(mode))
    pylab.ylim(0,5)
    #pylab.ylim(0,LLHCap)
    pylab.tight_layout()
    pylab.xlim(-.2,.2)
    pylab.grid()
else:
    print("file not found")

## PPC Run-2 (with a finer scan over modes 1,2,4)
delete previously created files for these modes (only run/ rerun previous code for mode 0,3,5)
For mode 1,2,4:

In [ ]:
#Parsing output from run 2: 

In [ ]:
Params=["Amp","Phs"]     #Forier Parameter (eg. Amp or Phs)
modes=[1,2,4,]       # mode (eg. 0,1,2,3...) no mode '0' for Phs
for Param in Params:
    for Param_num in modes:
        outdir='/data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2_run_2/'+str(Param)+"/"+str(Param_num)+"/out/"
        outfile='/data/user/anegi/ppc/LLHs/spice_ftp-v2_run_2/'+str(Param)+"_"+str(Param_num)+".txt"
        Processoutput(outdir,outfile)        

In [5]:
#Getting widths from run 2:

## Correlated Params:

In [7]:
def Processoutput(rootdir, outpath):
    count=0
    PrintEvery=100
    fit_dirs = []
    fit_files = glob.glob(rootdir + "/out*")
    #n=1

    if not fit_files :
        print("WARNING: No Fit Directories Found In " + str(rootdir) )
        return
    else:
        print("Found "  + str(len(fit_files)) + " Fit Files....")


    outfile = open(outpath, "a")

    print("Writing Output to: "  + str(outfile))
    outfile.write("#Params_mode1_mode2  shift DOM LLH \n")

    for fit_filen in fit_files:
        try:
            fit_file = open(str(fit_filen),'r')
        except:
            print("WARNING: " + str(fit_filen) + " Does Not Exist!" )
        else:
            try:
                lines= fit_file.readlines()     
            except:
                print("WARNING: " + str(fit_filen) + " Is Empty!" )
                break
            HeaderLine=lines[2]
            #print(HeaderLine)
            Param=HeaderLine.split("_")[0].split(" ")[1]
            ParamNum1=HeaderLine.split("_")[1]
            ParamNum2=HeaderLine.split("_")[2]
            try:
                shift=float(HeaderLine.split("_")[3].replace(".dat",""))
                #print("sh",shift)
            except:
                print("Shift not found")
            for line in lines[3:-1]:
                if "DOM " in line:
                    if not "Bad" in line:
                        DOM=int(line.split()[1])
                        
                if 'LLH' in line:
                    try:
                    #print(LLH,shift,DOM,fit_filen)
                        LLH=re.findall("LLH=\d+\.\d+", line)[0][4:]
                    except:
                        LLH=re.findall("LLH=\d", line)[0][4:]
                        print(LLH,shift,DOM,fit_filen)
                        #LLH=re.findall("LLH=\d", line)[0][4:]
                        #print(LLH)
                    
                    #DT=line.split("=")[2][0:-3]
                    if(DOM!=0):
                        writeline=str(Param)+"_"+str(ParamNum1)+"_"+str(ParamNum2)+" " +str(shift)+ " " + str(DOM) + " " + str(LLH) 
                        outfile.write(writeline + "\n")                    
                    count=count+1
                    #if(count%PrintEvery==0):
                        #print("Absorbed "+str(count) + " LLHs \n")
        fit_file.close()
        #if n==1:
           # break
    outfile.close()

    print("Done!")

In [ ]:
#AmpPhs:
Processoutput("/data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/AmpPhs/out/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/AmpPhs.txt")

In [8]:
#AmpPhs:
Processoutput("/data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/AmpPhs/out/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/AmpPhs.txt")

Found 750 Fit Files....
Writing Output to: <_io.TextIOWrapper name='/data/user/anegi/ppc/LLHs/spice_ftp-v2/AmpPhs.txt' mode='a' encoding='UTF-8'>
1 0.3333 47 /data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/AmpPhs/out/out_125
1 3.0 58 /data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/AmpPhs/out/out_749
Done!


In [9]:
#AmpAmp:
Processoutput("/data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/AmpAmp/out/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/AmpAmp.txt")

Found 300 Fit Files....
Writing Output to: <_io.TextIOWrapper name='/data/user/anegi/ppc/LLHs/spice_ftp-v2/AmpAmp.txt' mode='a' encoding='UTF-8'>
Done!


In [10]:
#PhsPhs:
Processoutput("/data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/PhsPhs/out/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/PhsPhs.txt")

Found 300 Fit Files....
Writing Output to: <_io.TextIOWrapper name='/data/user/anegi/ppc/LLHs/spice_ftp-v2/PhsPhs.txt' mode='a' encoding='UTF-8'>
1 0.3333 35 /data/user/anegi/ppc/condor/ppc_logs/spice_ftp-v2/PhsPhs/out/out_184
Done!


In [ ]:
#Updated to remove outliers:

def LLHWidths(infile,outdir_llh,outdir_widths,Bad_Doms=[]):
    dat=np.loadtxt(infile,dtype=str)
    params=np.unique(dat[:,0])
    #print(params)
    #params=['AmpPhs_0_5']
    for param in params:
        
        LLHDict=dict()
        LLHDict[str(param)]=dict()
        mask1=(dat[:,0]==str(param))
        dat2=dat[mask1,1:]        
        shifts=np.unique(dat2[:,0])
        #print(len(shifts))
        #pairs=[0,0]
        for shift in shifts:
            mask=(dat2[:,0]==shift)
            subdat=dat2[mask,1:]         #subarray for a particular shift value
            LLH_shift=0
            BadDom=[]        
            for i in range(1,61):
                if i not in Bad_Doms:
                    try:
                        LLH_dom=float(subdat[subdat[:,0]==str(i),1])      #DoM=i  ,LLH(i)=LLH_dom
                        #print("dom",LLH_dom,i)
                        LLH_shift+=LLH_dom
                        #print("total",LLH_shift)
                    except:
                        #print("shift ",shift,"missing LLH: DoM",i)
                        BadDom.append(i)
                        pass        
            if len(BadDom)<2:
                LLH_shift=LLH_shift*(60-len(Bad_Doms))/(60-len(Bad_Doms)-len(BadDom))                   #rescaling total LLH if only missing 1 DoM
                LLHDict[str(param)][float(shift)]=float(LLH_shift)
            else:                                #ignoring the shift if missing LLh for more than 1 DoM (rerun jobs for missing DoMs if required)
                print("shift ",shift,"missing LLH: DoM",BadDom)           
#        print(LLHDict)
        width=[]
        for Param in LLHDict.keys():
            Shifts_dict=LLHDict[Param]
            shift_llh=np.array(list(Shifts_dict.items()))
            #print(shift_llh)
            if os.path.isfile(str(outdir_llh)+"/"+str(Param.split("_")[0])+"/"+str(Param.split("_")[0])+"_"+str(Param.split("_")[1])+"_"+str(Param.split("_")[2])+"_llh.txt")==0:
                print("creating new llh file for ",Param)
                np.savetxt(str(outdir_llh)+"/"+str(Param.split("_")[0])+"/"+str(Param.split("_")[0])+"_"+str(Param.split("_")[1])+"_"+str(Param.split("_")[2])+"_llh.txt",shift_llh,delimiter=" ")
            else:
                print("llh file already exists")

            pairs_0=[]                
            for i in shift_llh:
                pairs_0.append([i[0],i[1]])
        
            
            #This is looped to get rid of any shift having having way too low llh value:(maximum one outlier)
            n=0
            for n in range(0,3):
                pairs=[]
                llh_min=min(np.array(pairs_0)[:,1])
                #print("min_llh:",llh_min,",  iteration:",n)
                for i in range(0,len(pairs_0)):
                    #print(np.array(pairs_0)[i,1],"cfgvh")
                    if np.array(pairs_0)[i,1]<llh_min+5:                      #only taking shifts withh LLH<5 for the curve fitting                   
                        pairs.append([pairs_0[i][0],pairs_0[i][1]-llh_min])
                #print("pairs",pairs,"iteration",n)                
                if len(pairs)>=5:                                        
                    break
                else:                                                        #if no points found close to llh min remove shift for llh min (might be wrong value)
                    for i in range(0,len(pairs_0)):
                        if (pairs_0[i][1])==float(llh_min):
                            llh_err=pairs_0[i]
                    pairs_0.remove(llh_err)
                    n+=1  
                    print("Warning: removing min llh shift",Param)
            if n==3:                                                      
                print("no llhs found below threshold",Param)
                continue
                
                 
            FittedCurve=polynomial.polyfit(np.array(pairs)[:,0],np.array(pairs)[:,1],2)        #fitting to a quadratic polynomial
            vals=np.arange(-3,3,0.001)
            minval=np.min(polynomial.polyval(vals,FittedCurve))
            PlusOneSig=(-FittedCurve[1]+np.sqrt(FittedCurve[1]**2-4.*(-0.5-minval+FittedCurve[0])*FittedCurve[2]))/(2.*FittedCurve[2])
            MinusOneSig=(-FittedCurve[1]-np.sqrt(FittedCurve[1]**2-4.*(-0.5-minval+FittedCurve[0])*FittedCurve[2]))/(2.*FittedCurve[2])
            Zero=(PlusOneSig+MinusOneSig)/2.
            if str(Param.split("_")[1])==str(0):
                print(PlusOneSig)
            if os.path.isfile(str(outdir_widths)+"/"+str(Param.split("_")[0])+"_widths.txt")==1:
                outfile = open(str(outdir_widths)+"/"+str(Param.split("_")[0])+"_widths.txt", "r")
                lines= outfile.readlines()
                count=0
                for line in lines:
                   # print(line[0],line[2])
                    if line[0]==str(Param.split("_")[1]) and line[2]==str(Param.split("_")[2]):  
                        count+=1
                outfile.close()        
                if count!=0:
                    print("Parameter width already exists: ", Param)
                else:
                    print("writing Param width: ",Param)
                    outfile = open(str(outdir_widths)+"/"+str(Param.split("_")[0])+"_widths.txt", "a")
                    writeline=str(Param.split("_")[1])+" "+str(Param.split("_")[2])+" "+str(Zero)+" "+str(MinusOneSig)+" "+str(PlusOneSig)                
                    outfile.write(writeline + "\n")
                    outfile.close()
            else: 
                print("creating new width file: ", str(outdir_widths)+"/"+str(Param.split("_")[0])+"_widths.txt")
                outfile = open(str(outdir_widths)+"/"+str(Param.split("_")[0])+"_widths.txt", "w")
                writeline="# Param1 Param2 central -1sig +1sig"                
                outfile.write(writeline + "\n")
                writeline=str(Param.split("_")[1])+" "+str(Param.split("_")[2])+" "+str(Zero)+" "+str(MinusOneSig)+" "+str(PlusOneSig)                
                outfile.write(writeline + "\n")
                outfile.close()


In [106]:
#AmpPhs

LLHWidths("/data/user/anegi/ppc/LLHs/spice_ftp-v2/AmpPhs.txt","/data/user/anegi/ppc/LLHs/spice_ftp-v2/llhs/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/widths/",Bad_Doms=[])



llh file already exists
-0.15490760508542467
creating new width file:  /data/user/anegi/ppc/LLHs/spice_ftp-v2/widths//AmpPhs_widths.txt


llh file already exists
0.6876543907948705
writing Param width:  AmpPhs_0_2


llh file already exists
1.4719006982076583
writing Param width:  AmpPhs_0_3


llh file already exists
1.2840618504941188
writing Param width:  AmpPhs_0_4


llh file already exists
0.8019939359579952
writing Param width:  AmpPhs_0_5


llh file already exists
writing Param width:  AmpPhs_1_1


llh file already exists
writing Param width:  AmpPhs_1_2


llh file already exists
writing Param width:  AmpPhs_1_3


llh file already exists
writing Param width:  AmpPhs_1_4


llh file already exists
writing Param width:  AmpPhs_1_5


llh file already exists
writing Param width:  AmpPhs_2_1


llh file already exists
writing Param width:  AmpPhs_2_2


llh file already exists
writing Param width:  AmpPhs_2_3


llh file already exists
writing Param width:  AmpPhs_2_4


llh file already 

In [ ]:
#AmpAmp

LLHWidths("/data/user/anegi/ppc/LLHs/spice_ftp-v2/AmpAmp.txt","/data/user/anegi/ppc/LLHs/spice_ftp-v2/llhs/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/widths/",Bad_Doms=[])

In [ ]:
#PhsPhs

LLHWidths("/data/user/anegi/ppc/LLHs/spice_ftp-v2/PhsPhs.txt","/data/user/anegi/ppc/LLHs/spice_ftp-v2/llhs/","/data/user/anegi/ppc/LLHs/spice_ftp-v2/widths/",Bad_Doms=[])